# Get Programming with F# by [Isaac Abraham](https://github.com/isaacabraham)

## “Fixing the billion-dollar mistake”

The post-2018 news for the handling of nulls in C# is that _null parameter checking_ (or _Parameter Null Checking_) is still [a proposal](https://github.com/dotnet/csharplang/blob/main/proposals/param-nullchecking.md): it did not make it to C# 10. This proposed feature is on top of other null-related features in C# such as:

- “`!` (null-forgiving) operator” [📖 [docs](https://docs.microsoft.com/en-us/dotnet/csharp/language-reference/operators/null-forgiving)]
- “Nullable reference types” [📖 [docs](https://docs.microsoft.com/en-us/dotnet/csharp/nullable-references)]
- “`??` and `??=` operators” [📖 [docs](https://docs.microsoft.com/en-us/dotnet/csharp/language-reference/operators/null-coalescing-operator)]
- “Null-conditional operators `?.` and `?[]`” [📖 [docs](https://docs.microsoft.com/en-us/dotnet/csharp/language-reference/operators/member-access-operators#null-conditional-operators--and-)]
- “Nullable value types” [📖 [docs](https://docs.microsoft.com/en-us/dotnet/csharp/language-reference/builtin-types/nullable-value-types)]

>Even members of the C# complier team have said that they regret putting null into the language. The problem is that now it’s in, it’s almost impossible to remove. …The simple truth is that the .NET type system itself isn’t geared toward allowing us to easily and consistently reason about mandatory and optional data.

In the world of C#, there is a project called _Optional_ [[GitHub](https://github.com/nlkl/Optional)] [📦 [NuGet](https://www.nuget.org/packages/Optional/)] led by [Nils Lück](https://www.linkedin.com/in/nils-l%C3%BCck-a2258358/?originalSubdomain=dk). The name of this C# project is clearly a direct reference to what is built from the ground up in F#: the `Option` type [📖 [docs](https://docs.microsoft.com/en-us/dotnet/fsharp/language-reference/options)], supported by the [Option Module](https://fsharp.github.io/fsharp-core-docs/reference/fsharp-core-optionmodule.html).

## the `Option` type is a discriminated union of `Some` and `None`

Lessons [17](https://github.com/BryanWilhite/jupyter-central/blob/master/get-programming-with-f-sharp/17-maps-dictionaries-and-sets.ipynb), [18](https://github.com/BryanWilhite/jupyter-central/blob/master/get-programming-with-f-sharp/18-folding-your-way-to-success.ipynb), [19](https://github.com/BryanWilhite/jupyter-central/blob/master/get-programming-with-f-sharp/19-capstone-03.ipynb) and [20](https://github.com/BryanWilhite/jupyter-central/blob/master/get-programming-with-f-sharp/20-program-flow-in-f-sharp.ipynb) use `None` which is part of the `Option` type, a discriminated union [[GitHub](https://github.com/dotnet/fsharp/blob/main/src/fsharp/FSharp.Core/prim-types.fsi#L2381)]:

```fsharp
type Option<'T> =
    | None: 'T option
    | Some: Value:'T -> 'T option
```

Lessons [18](https://github.com/BryanWilhite/jupyter-central/blob/master/get-programming-with-f-sharp/18-folding-your-way-to-success.ipynb), [19](https://github.com/BryanWilhite/jupyter-central/blob/master/get-programming-with-f-sharp/19-capstone-03.ipynb) and [20](https://github.com/BryanWilhite/jupyter-central/blob/master/get-programming-with-f-sharp/20-program-flow-in-f-sharp.ipynb) use `Some`.

`Option` is used to explicitly express the _opposite_ of the default in F#:

>…all F# types (tuples, records, and discriminated unions) behave in the same way, in that they’re _mandatory by default_.

`Option` allows us to diverge from this default, making _optional types_ (called _Maybe types_ in other languages).

## `Option` can be used to protect against `null`

Instead of implicitly formalizing a relationship with the unknown by allowing `null` to propagate throughout, we can express optionality instead with something like `toStringOption` that can be used as barrier against the outside world (the greater .NET), represented by `someFunction`:

In [ ]:
#!fsharp

open System

let someFunction =
    let strings = [ "one"; null; "three"; ""; "  " ]
    let index = Random().Next(strings.Length)
    strings |> List.item index

let toStringOption (s: string) =
    match String.IsNullOrWhiteSpace(s) with
    | false -> Some s
    | true -> None

someFunction |> toStringOption |> printf "%A"

None

## the Option Module can be used to express complex rules based on optional values

The Option Module [[GitHub](https://fsharp.github.io/fsharp-core-docs/reference/fsharp-core-optionmodule.html)] functions look like those for F# collections. For example, we can use `Option.count` to convert strings from `someFunction` to `1` or `0`:

In [ ]:
#!fsharp

someFunction |> toStringOption |> Option.count |> printf "%A"

0

Instead of `1` or `0`, we can get `true` or `false` with `Option.isSome`:

In [ ]:
#!fsharp

someFunction |> toStringOption |> Option.isSome |> printf "%A"

false

We can use `Option.defaultWith` to define an alternate value for `None`:

In [ ]:
#!fsharp

someFunction
|> toStringOption
|> Option.defaultWith (fun () -> "<empty>")
|> printf "%A"

"<empty>"

We see a technique here for removing `null` values:

1. convert `null` _to_ `Option`
2. use `Option.defaultWith` to convert _from_ `Option`

Step _2_ is important because passing around `Option` means writing `Some` a whole bunch of times. Take this function, `calculateAnnualPremiumUsd`, from page 263:

In [ ]:
#!fsharp

let calculateAnnualPremiumUsd score =
    match score with
    | Some 0 -> 250
    | Some score when score < 0 -> 400
    | Some score when score > 0 -> 150
    | None ->
        printfn "No score supplied! Using temporary premium."
        300

[Some 250 ; None]
    |> List.iter (fun score -> score |> calculateAnnualPremiumUsd |> printfn "%A")


150
No score supplied! Using temporary premium.
300


Instead of using `Some` _three_ times, we can use `Some` _once_ and nest `match`:

In [ ]:
#!fsharp

let calculateAnnualPremiumUsd score =

    match score with
    | Some n ->
        match n with
        | 0 -> 250
        | n when n < 0 -> 400
        | n when n > 0 -> 150
    | None ->
        printfn "No score supplied! Using temporary premium."
        300

[Some 250 ; None]
    |> List.iter (fun score -> score |> calculateAnnualPremiumUsd |> printfn "%A")

150
No score supplied! Using temporary premium.
300


## “Option Module enables Railway Oriented Programming”

The `match`-`Some`-`None` pattern in the code sample above can be regarded as an example of _Railway Oriented Programming_, according to “[Option Module enables Railway Oriented Programming](https://sodocumentation.net/fsharp/topic/3175/option-types#option-module-enables-railway-oriented-programming).” However, when we search the classic [_F# for Fun and Profit_ article by Scott Wlaschin on railway programming](https://fsharpforfunandprofit.com/rop/), `Option` is not even mentioned! (The GitHub site backing _F# for Fun and Profit_ does [prove that it is aware](https://github.com/swlaschin/fsharpforfunandprofit.com/search?l=Markdown&p=2&q=option) of the existence of `Option`.)

Railway Oriented Programming is more about error handling (the equivalent of `try`-`catch` blocks in C#) which somehow leads to the subject matter of the [Kleisli category](https://en.wikipedia.org/wiki/Kleisli_category) (where category refers to _category theory_).

## `Option.map` eliminates the need to explicitly match `Some` and `None`

The opinion here is that `Option.map` is the most important function in the Options Module. When used with `Option.defaultWith`, we can make another railway where the `mapping` argument is already converted _from_ `Option` (and `Some` is completely implied/invisible):

In [ ]:
#!fsharp

let calculateAnnualPremiumUsd score =
    let mapping n =
        match n with
        | 0 -> 250
        | n when n < 0 -> 400
        | n when n > 0 -> 150

    score
    |> Option.map mapping
    |> Option.defaultWith (fun () ->
        printfn "No score supplied! Using temporary premium."
        300)

[Some 250 ; None]
    |> List.iter (fun score -> score |> calculateAnnualPremiumUsd |> printfn "%A")

150
No score supplied! Using temporary premium.
300


@[BryanWilhite](https://twitter.com/BryanWilhite)


In [ ]:
#!about

.NET Interactive© 2020 Microsoft CorporationVersion: 1.0.258004+3429f67626d2f15eec7d8f15a29215909284cb79Build date: 2021-12-01T01:48:00.0000000Zhttps://github.com/dotnet/interactive
